<a href="https://colab.research.google.com/github/edurso/obj-detect/blob/master/notebooks/tensorrt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convert TensroFlow `saved_model` to [TensorRT](https://github.com/NVIDIA/TensorRT/tree/main/samples/python/tensorflow_object_detection_api) Engine

define model location

In [1]:
workdir = '/content/workspace/MyDrive/tensorflow/power-port-targeting/exported_models/trained_model'
mname = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
from google.colab import drive
import os
!mkdir -p /content/workspace
drive.mount('/content/workspace')

Mounted at /content/workspace


download `trained_model` from google drive to `workdir`

check python version

In [2]:
!python3 --version

Python 3.7.13


## Installation & Environment Configuration

install tfod api

In [3]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models /content/models

if "utils" in pathlib.Path.cwd().parts:
  while "utils" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('utils').exists():
  !git clone --depth 1 https://github.com/edurso/obj-detect /content/utils

Cloning into '/content/models'...
remote: Enumerating objects: 3344, done.
remote: Counting objects: 100% (3344/3344), done.
remote: Compressing objects: 100% (2780/2780), done.
remote: Total 3344 (delta 882), reused 1390 (delta 513), pack-reused 0
Receiving objects: 100% (3344/3344), 34.34 MiB | 26.28 MiB/s, done.
Resolving deltas: 100% (882/882), done.
Cloning into '/content/utils'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 22 (delta 0), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [4]:
%%bash

# update pip
pip3 install --upgrade pip

# check opencv dependency versions
pip3 uninstall opencv-python-headless==4.5.5.62 
pip3 install opencv-python-headless==4.1.2.30
pip3 install onnx-graphsurgeon==0.3.10 --index-url https://pypi.ngc.nvidia.com

# compile protos and install object_detection
cd /content/models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install --use-feature=2020-resolver .

  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 30.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.ngc.nvidia.com
Processing /content/models/research
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 66.5 MB/s eta 0:00:00
     ━━

ERROR: Could not find a version that satisfies the requirement onnx (from onnx-graphsurgeon) (from versions: none)
ERROR: No matching distribution found for onnx
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.12.2 requires dill>=0.3.4, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.0.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [5]:
!pip show object-detection

Name: object-detection
Version: 0.1
Summary: Tensorflow Object Detection Library
Home-page: UNKNOWN
Author: 
Author-email: 
License: UNKNOWN
Location: /usr/local/lib/python3.7/dist-packages
Requires: apache-beam, avro-python3, contextlib2, Cython, keras, lvis, lxml, matplotlib, pandas, pillow, pycocotools, scipy, six, tensorflow-io, tf-models-official, tf-slim
Required-by: 


install tensorrt

In [6]:
!nvidia-smi

Tue May 17 19:38:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
ubuntu='ubuntu1804'
tag='cuda11.4-trt8.2.5.1-ga-20220505'
os.chdir('/content/workspace/MyDrive/tensorflow/')
!dpkg -i nv-tensorrt-repo-{ubuntu}-{tag}_1-1_amd64.deb
!apt-key add /var/nv-tensorrt-repo-{ubuntu}-{tag}/*.pub
!apt-get install python3-libnvinfer-dev

!python3 -m pip install --upgrade setuptools pip
!python3 -m pip install nvidia-pyindex
!python3 -m pip install pycuda 

!apt-get update

Selecting previously unselected package nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505_1-1_amd64.deb ...
Unpacking nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505 (1-1) ...
Setting up nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505 (1-1) ...

The public CUDA GPG key does not appear to be installed.
To install the key, run this command:
sudo apt-key add /var/nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505/82307095.pub

OK
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libnvinfer-dev libnvinfer-plugin-dev libnvinfer-plugin8 l

In [8]:
!apt-get install tensorrt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libnvinfer-bin libnvinfer-doc libnvinfer-samples
The following NEW packages will be installed:
  libnvinfer-bin libnvinfer-doc libnvinfer-samples tensorrt
0 upgraded, 4 newly installed, 0 to remove and 74 not upgraded.
Need to get 0 B/467 MB of archives.
After this operation, 583 MB of additional disk space will be used.
Get:1 file:/var/nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505  libnvinfer-bin 8.2.5-1+cuda11.4 [146 kB]
Get:2 file:/var/nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505  libnvinfer-doc 8.2.5-1+cuda11.4 [7,178 kB]
Get:3 file:/var/nv-tensorrt-repo-ubuntu1804-cuda11.4-trt8.2.5.1-ga-20220505  libnvinfer-samples 8.2.5-1+cuda11.4 [460 MB]
Ge

In [9]:
!dpkg -l | grep TensorRT

ii  libnvinfer-bin                                              8.2.5-1+cuda11.4                    amd64        TensorRT binaries
ii  libnvinfer-dev                                              8.2.5-1+cuda11.4                    amd64        TensorRT development libraries and headers
ii  libnvinfer-doc                                              8.2.5-1+cuda11.4                    all          TensorRT documentation
ii  libnvinfer-plugin-dev                                       8.2.5-1+cuda11.4                    amd64        TensorRT plugin libraries
ii  libnvinfer-plugin8                                          8.2.5-1+cuda11.4                    amd64        TensorRT plugin libraries
ii  libnvinfer-samples                                          8.2.5-1+cuda11.4                    all          TensorRT samples
ii  libnvinfer8                                                 8.2.5-1+cuda11.4                    amd64        TensorRT runtime libraries
ii  libnvonnxparsers-dev     

In [10]:
import tensorrt
print(tensorrt.__version__)

ModuleNotFoundError: ignored

get tensorrt utility files

In [ ]:
url = 'https://raw.githubusercontent.com/NVIDIA/TensorRT/main/samples/python/tensorflow_object_detection_api/{}.py'
files = ['build_engine', 'create_onnx', 'infer', 'compare_tf', 'infer', 'onnx_utils', 'visualize']
os.chdir('/content/utils/scripts')
for file in files:
    furl = url.format(file)
    !wget {furl}

## Create TensorRT Engine

In [ ]:
os.chdir('/content/utils/scripts')
!python3 create_onnx.py \
    --pipeline_config {workdir}pipeline.config \
    --saved_model {workdir}saved_model \
    --onnx {workdir}model.onnx

In [ ]:
os.chdir('/content/utils/scripts')
!python3 build_engine.py \
    --onnx {workdir}model.onnx \
    --engine {workdir}engine.trt \
    --precision fp16

## Test TensorRT Engine

In [ ]:
!trtexec \
    --loadEngine={workdir}engine.trt \
    --useCudaGraph --noDataTransfers \
    --iterations=100 --avgRuns=100

In [ ]:
os.chdir('/content/utils/scripts')
!python3 infer.py \
    --engine {workdir}engine.trt \
    --input ../img \
    --output {workdir}/output \
    --preprocessor fixed_shape_resizer \
    --labels {workdir}label_map.txt

## Clean Up

In [ ]:
os.chdir('/content/utils/scripts')
for file in files:
    fname = '{}.py'.format(file)
    !rm -rf {fname}